In [1]:
import pandas as pd

In [6]:
df = pd.read_csv(r'./dataset.csv')

df2 = df[["track_id","writers"]]
df2_dict = df2.to_dict(orient='records')

for d in df2_dict:
    d["writers"] = d["writers"].split("-")

edges = ""
for d in df2_dict:
    for f in df2_dict:
        if d["track_id"] != f["track_id"]:
            for d2 in d["writers"]:
                if d2 in f["writers"]:
                    edges += str(d["track_id"]) + " " + str(f["track_id"]) + '\n'
strE = '\n'.join(list(set(list(edges.split('\n')))))
outfile = open("edges_list.txt","w")
outfile.write(strE)
outfile.close()

AttributeError: 'list' object has no attribute 'join'